<a href="https://colab.research.google.com/github/thestbobo/DLML_FL_project/blob/modularizing-mask-computation/colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment Setup**

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Clone GitHub repository
!git clone -b modularizing-mask-computation https://github.com/thestbobo/DLML_FL_project.git
%cd DLML_FL_project

Cloning into 'DLML_FL_project'...
remote: Enumerating objects: 269, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 269 (delta 94), reused 120 (delta 56), pack-reused 102 (from 1)
Receiving objects: 100% (269/269), 67.41 KiB | 556.00 KiB/s, done.
Resolving deltas: 100% (129/129), done.
/content/DLML_FL_project


In [1]:
# Install dependencies
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
%cd DLML_FL_project


[Errno 2] No such file or directory: 'DLML_FL_project'
/content/DLML_FL_project


**Pull after local commits**

In [ ]:
!git pull origin modularizing-mask-computation


remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 17 (delta 12), reused 12 (delta 7), pack-reused 0 (from 0)
Unpacking objects: 100% (17/17), 1.96 KiB | 400.00 KiB/s, done.
From https://github.com/thestbobo/DLML_FL_project
 * branch            main       -> FETCH_HEAD
   d25564d..91fe87f  main       -> origin/main
Updating d25564d..91fe87f
Fast-forward
 .idea/misc.xml      |  2 +-
 fl_core/__init__.py |  0
 requirements.txt    |  3 ++-
 train.py            |  2 +-
 train_federated.py  | 28 ++++++++++++++++------------
 5 files changed, 20 insertions(+), 15 deletions(-)
 delete mode 100644 fl_core/__init__.py


**Environment check**

In [4]:
import torch
print("CUDA avaiable:", torch.cuda.is_available())

CUDA avaiable: True


**Load Configuration (hyperparameters) and Paths (in/out)**

-Example file for config.yaml

batch_size: 64

lr: 0.01

-Example of code usage:

print(config["batch_size"]) --> 64


In [6]:
import yaml
from pathlib import Path

# load YAML config
with open("config/config.yaml") as f:
  config = yaml.safe_load(f)

# Setup paths and device
DATA_DIR = Path("./data")
DEVICE= torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(config)

{'batch_size': 256, 'val_split': 0.2, 'num_workers': 4, 'learning_rate': 0.008805, 'weight_decay': 0.01, 'momentum': 0.9, 't_max': 50, 'epochs': 50, 'checkpoint_path': '/content/drive/MyDrive/DL_project/checkpoints/checkpoint_35.pth', 'out_checkpoint_dir': '/content/drive/MyDrive/DL_project/checkpoints', 'NUM_CLIENTS': 100, 'CLIENT_FRACTION': 0.1, 'LOCAL_EPOCHS': 4, 'BATCH_SIZE': 32, 'LR': 0.01, 'ROUNDS': 100, 'IID': True, 'NC': None, 'SEED': 42}


**Training and results visualisazion**

In [7]:
%env WANDB_API_KEY=9efaa9e72ef3b211a9239aed9b1ea7dcb2f680ad

env: WANDB_API_KEY=9efaa9e72ef3b211a9239aed9b1ea7dcb2f680ad


The code snippet below runs an already created sweep on the centralized model. If you want to run the sweep yourself and view the results, you need to create a new sweep via terminal, copy the given sweep_id and change it below

In [ ]:
import wandb
import importlib.util
import shutil
from pathlib import Path


# setup sweep function dynamically
spec = importlib.util.spec_from_file_location("train", "/content/DLML_FL_project/train.py")
train_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(train_module)
main = train_module.main


# my wandb api key
%env WANDB_API_KEY=e78d3761af2b7e3a53b7a9ba6385370820743648


# run sweep - change here to run it yourself
sweep_id = "s297466-politecnico-di-torino/DLML_FL_project/fx50wubp"
wandb.agent(sweep_id, function=main, count=5)

CENTRALIZED

In [11]:
!python train.py

Using device:  cuda
wandb: Currently logged in as: s297466 (s297466-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.10
wandb: Run data is saved locally in /content/DLML_FL_project/wandb/run-20250501_100731-zble4yuy
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run icy-wood-80
wandb: ⭐️ View project at https://wandb.ai/s297466-politecnico-di-torino/CIFAR-100_centralized
wandb: 🚀 View run at https://wandb.ai/s297466-politecnico-di-torino/CIFAR-100_centralized/runs/zble4yuy
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/f

FEDERATED

In [ ]:
!python train_federated.py